In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
import re
import seaborn as sns
sns.set(color_codes=True)

# pip install tabula-py
import tabula

In [2]:
df_pdf = tabula.read_pdf('kcp-mag-filialu-2021.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)
df.head(10)

searchfor = ['Направление','Общие',"очное"]
mask = df[1].str.contains('|'.join(searchfor), na=False)|df[2].str.contains('|'.join(searchfor), na=False)
df = df[~mask][[0,1]].copy()


mask_uslovie = df[0].str.contains("УСЛОВИЯ", na=False)


#df[0] = df[0].str.replace('УСЛОВИЯ ПОСТУПЛЕНИЯ:\r','')
df[0] = df[0].str.replace('УСЛОВИЯ ПОСТУПЛЕНИЯ:\r','')
df[0] = df[0].map(lambda x: x.split('– программы')[0]) # внимание тут разное тире в разных строках!!! 
df[0] = df[0].map(lambda x: x.split('- программы')[0]) # внимание тут разное тире в разных строках!!! 
df[0] = df[0].map(lambda x: x.split('- филиал')[0])
mask_list = np.array(df[mask_uslovie].index)


# with pd.option_context('display.max_rows', None):
#          display(df)
df.head(3)

,0,1
3,Алтайский филиал,NaN
4,07.04.04,Градостроительство\rпо программе магистратуры ...
5,37.04.01,Психология\rпо программе магистратуры в рамках...


In [4]:
def func_dic(df_test):
    dic = {} 
    code = df_test[0]
    dic.update({0: code})
    tmp = df_test[1]
    napravlenie = tmp.split('\r',1)[0]
    dic.update({1: napravlenie})
    tmp = tmp.split('\r',2)[2]
    tmp = " ".join(tmp.split())
    tmp = re.split('\d.',tmp)[1:]
    tmp = np.char.replace(tmp,'\r',' ')
    tmp = np.char.replace(tmp,'.','')
    tmp = np.char.strip(tmp)
    dic.update({2: tmp})
    df_tmp = pd.DataFrame(dic)
    return df_tmp


df_multi_institute = []
mask_shape = mask_list.shape[0]
for i in range(mask_shape):
    if i < mask_shape-1:
        end = mask_list[i+1]-1
    else:
        end = df.index.max()
    
    df_fil = df.loc[mask_list[i]:end]
    
    tmp = [ func_dic(df_fil.loc[idx]) for idx in df_fil.iloc[1:].index.values]    
    df_multi = pd.concat(tmp, ignore_index=True)    
    df_multi[3] = df_fil.iloc[0][0].strip()

    
    df_multi = df_multi.drop_duplicates()
    df_multi_institute.append(df_multi)

df_multi_institute = pd.concat(df_multi_institute, ignore_index=True) 
df_multi_institute.columns = ['код', 'стандарт', 'программа', 'подразделение']

df_multi_institute

,код,стандарт,программа,подразделение
0,07.04.04,Градостроительство,Современная урбанистика,Алтайский филиал
1,37.04.01,Психология,Прикладная социальная психология,Алтайский филиал
2,38.04.01,Экономика,Экономика здравоохранения,Алтайский филиал
3,38.04.04,Государственное и муниципальное управление,Система государственного и муниципального упра...,Алтайский филиал
4,38.04.04,Государственное и муниципальное управление,Управление и экономика в сфере здравоохранения,Алтайский филиал
...,...,...,...,...
123,40.04.01,Юриспруденция,Судебная власть; прокурорский надзор; организа...,Южно-Российский институт управления
124,40.04.01,Юриспруденция,"Уголовное право; криминология, уголовно-исполн...",Южно-Российский институт управления
125,40.04.01,Юриспруденция,Административное право; финансовое право,Южно-Российский институт управления
126,40.04.01,Юриспруденция,Конституционное право; муниципальное право,Южно-Российский институт управления
